<a href="https://colab.research.google.com/github/kevinOriginal/news-recommender/blob/main/Big_data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from glob import glob
import pandas as pd
import pyarrow.parquet as pq

In [ ]:
import os, sys
import pandas as pd 
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel
import torch
import torch.nn as nn
import numpy as np

In [ ]:
# 구글 드라이브 사용 시 주석 해제
'''
from google.colab import drive
drive.mount('/content/mnt',force_remount=True)

# Path에, 구글드라이브 내의 데이터 경로 설정
path = '/content/mnt/MyDrive/colab_data/big_data/user_vector.parquet'

'''

"\nfrom google.colab import drive\ndrive.mount('/content/mnt',force_remount=True)\n\n# Path에, 구글드라이브 내의 데이터 경로 설정\npath = '/content/mnt/MyDrive/colab_data/big_data/user_vector.parquet'\n\n"

In [ ]:
# GPU 할당 설정
GPU_NUM = 0 # 원하는 GPU 번호 입력
device = torch.device(f'cuda:{GPU_NUM}' if torch.cuda.is_available() else 'cpu')
torch.cuda.set_device(device) # change allocation of current GPU
print ('Current cuda device ', torch.cuda.current_device()) # check

# Additional Infos
if device.type == 'cuda':
    print(torch.cuda.get_device_name(GPU_NUM))
    print('Memory Usage:')
    print('Allocated:', round(torch.cuda.memory_allocated(GPU_NUM)/1024**3,1), 'GB')
    print('Cached:   ', round(torch.cuda.memory_cached(GPU_NUM)/1024**3,1), 'GB')



Current cuda device  0
GeForce RTX 2060 SUPER
Memory Usage:
Allocated: 0.0 GB
Cached:    0.0 GB


In [ ]:
# Read user_vector.parquet
path = "./data/user_vector.parquet"
df = pd.read_parquet(path) 

In [ ]:
#device = torch.device('cpu')
# Convert to torch 데이터를 텐서로 불러옴. cpu에 올리는 이유는 램 부족 때문
data = torch.DoubleTensor(df.values).to('cpu')
print("전체 데이터 : ", data.shape)

torch.Size([356295, 1422])

In [ ]:
# 메인 유저 설정 (추천 받는 사용자)

main_user = data[7].to(device)

tensor([0., 0., 0.,  ..., 0., 0., 0.], device='cuda:0', dtype=torch.float64)


In [ ]:
# 비슷한 사용자를 몇 명 뽑을 것인지.

num_k = 5

In [ ]:
# Cosine Similarity 
cos = nn.CosineSimilarity(dim= -1, eps=1e-6)

# 높은 Cos값, 인덱스 저장할 리스트
stored_index = []
stored_values = []

# VRAM 부족으로, 10만개씩 끊어서 계산
# 10만개 당 cos_similarity 높은 num_k개씩 뽑아서 저장
for i in range(0, int(data.shape[0]/100000)) : 
    torch.cuda.empty_cache()
    val = cos(main_user,data[100000 * i : 100000 *i + 100000].to(device))
    values, index = torch.topk(val,num_k)
    index = index + 100000*i
    stored_index.append(index)
    stored_values.append(values)
    torch.cuda.empty_cache()

### 최종적으로 num_k개 유저 추리기
#
# 텐서로 변환
stored_index = torch.cat(stored_index,0)
stored_values = torch.cat(stored_values,0)
print(stored_index,stored_values)

# 상위 5개 추리기
similar_values, index_of_stored_index = torch.topk(stored_values,num_k)

# 인덱스 변환
similar_index = []
for i in index_of_stored_index :
    print(i)
    similar_index.append(stored_index[i])
### 최종적으로 num_k개 유저 추리기 끝

tensor([     7,  74391,  58180,   3502,   1197, 177914, 190350, 110669, 192317,
        169183, 231119, 225375, 222501, 244744, 224252], device='cuda:0') tensor([1.0000, 0.4472, 0.4472, 0.3651, 0.3651, 0.4743, 0.4472, 0.4472, 0.4472,
        0.4472, 0.4743, 0.4472, 0.4472, 0.4472, 0.4472], device='cuda:0',
       dtype=torch.float64)
tensor(0, device='cuda:0')
tensor(10, device='cuda:0')
tensor(5, device='cuda:0')
tensor(2, device='cuda:0')
tensor(1, device='cuda:0')


In [ ]:
# num_k개 추린 유저들의 벡터를 similar_users에 저장
similar_users = []
for i in range(0,num_k) :
  user_id = df.index[similar_index[i]]
  similar_value = similar_values[i]
  print("Index : {:10d}, ID : {:10s}, Similar Value : {}".format(similar_index[i],user_id,similar_value))
  similar_users.append(df.loc[user_id])

similar_users = torch.DoubleTensor(k_rows)

Index :          7, ID : 00004p2pq , Similar Value : 1.0
Index :     231119, ID : noperCXP  , Similar Value : 0.4743416490252569
Index :     177914, ID : knig2u7aR , Similar Value : 0.4743416490252569
Index :      58180, ID : cwin25l1b , Similar Value : 0.4472135954999579
Index :      74391, ID : dpem3RPfE , Similar Value : 0.4472135954999579


In [ ]:
# 벡터 체크
similar_users.shape

torch.Size([5, 1422])

In [ ]:
# num_k명의 벡터 평균치 계산
similar_users_mean = torch.mean(similar_users,dim=0)

# 하나의 평균 벡터가 나옴
similar_users_mean.shape

torch.Size([1422])

In [ ]:
# num_k명의 평균 벡터와, 주인공 유저와의 벡터 차잇값을 계산
diffence_val = torch.abs(similar_users_mean - main_user.cpu())

In [ ]:
# 차이가 가장 많이 나는 값(뉴스) 추출. 차잇값, 인덱스 반환
num_recommend_news = 5 # 추천할 뉴스의 갯수
recommend_values, recommend_news_index = torch.topk(diffence_val,num_recommend_news)

In [ ]:
# 추출된 인덱스에 해당하는 뉴스는, 
# 주인공 사용자는 보지 않았지만 비슷한 사용자들은 본 뉴스들이다.
# 혹은, 주인공 사용자만 보고 비슷한 사용자들은 전부 보지 않은 뉴스들이다.

print("추천도 : {}".format(recommend_values))
print("추천 뉴스의 data에서의 Columns 인덱스 : {}".format(recommend_news_index))

추천도 : tensor([0.8000, 0.8000, 0.8000, 0.6000, 0.6000], dtype=torch.float64)
추천 뉴스의 data에서의 Columns 인덱스 : tensor([111, 487, 922, 673,  87])


In [ ]:
base_address = "https://news.naver.com/main/read.nhn?mode=LSD&mid=shm&sid1=102&oid={}&aid={}"
# 추천 뉴스 출력
for i in range(0, 5) :

  ids = df.columns[recommend_news_index[i].item()]
  oid = ids[4:7]
  aid = ids[-10:]
  #print(oid,aid)
  print (base_address.format(oid,aid))

https://news.naver.com/main/read.nhn?mode=LSD&mid=shm&sid1=102&oid=005&aid=0001308925
https://news.naver.com/main/read.nhn?mode=LSD&mid=shm&sid1=102&oid=022&aid=0003454387
https://news.naver.com/main/read.nhn?mode=LSD&mid=shm&sid1=102&oid=052&aid=0001425517
https://news.naver.com/main/read.nhn?mode=LSD&mid=shm&sid1=102&oid=025&aid=0002990918
https://news.naver.com/main/read.nhn?mode=LSD&mid=shm&sid1=102&oid=003&aid=0009801710


In [ ]:
# 참고. 유저가 본 뉴스 목록들
# user_index에 유저 인덱스 값을 넣으면 된다
user_index = 7
test_user_vector = torch.DoubleTensor(data[user_index])
for i in range(0, test_user_vector.shape[0]) :
  if test_user_vector[i] > 0 :
    ids = df.columns[i]
    oid = ids[4:7]
    aid = ids[-10:]
    #print(oid,aid)
    print (base_address.format(oid,aid))
    

https://news.naver.com/main/read.nhn?mode=LSD&mid=shm&sid1=102&oid=003&aid=0009801710
https://news.naver.com/main/read.nhn?mode=LSD&mid=shm&sid1=102&oid=005&aid=0001308925
https://news.naver.com/main/read.nhn?mode=LSD&mid=shm&sid1=102&oid=022&aid=0003454387
https://news.naver.com/main/read.nhn?mode=LSD&mid=shm&sid1=102&oid=025&aid=0002990918
https://news.naver.com/main/read.nhn?mode=LSD&mid=shm&sid1=102&oid=025&aid=0002992042
https://news.naver.com/main/read.nhn?mode=LSD&mid=shm&sid1=102&oid=047&aid=0002266102
https://news.naver.com/main/read.nhn?mode=LSD&mid=shm&sid1=102&oid=052&aid=0001425517
https://news.naver.com/main/read.nhn?mode=LSD&mid=shm&sid1=102&oid=081&aid=0003082022
https://news.naver.com/main/read.nhn?mode=LSD&mid=shm&sid1=102&oid=081&aid=0003082072
https://news.naver.com/main/read.nhn?mode=LSD&mid=shm&sid1=102&oid=437&aid=0000235831
